<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/main/ex13noteB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex13noteB

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2022)


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

----
## 次元削減(3) 主成分分析 - 後編
----




---
### 変換後の特徴の分散共分散行列

主成分分析で定まる変換を行って得られる特徴量の性質について説明します．

#### 変換後の分散共分散行列は対角行列になる

$N$個の$D$次元ベクトルから成る学習データ
$$
\{ \mathbf{x}_n \in {\cal R}^{D} | n = 1, 2, \ldots, N\}
$$
が与えられるとします．ただし，このデータの平均は $\mathbf{0}$ とします．$\mathbf{x}$ の分散共分散行列を $V_x$ と書くことにすると，平均が $\mathbf{0}$ なので，
$$
V_x = \frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_n\mathbf{x}_n^{\top}
$$
となります．$V_x$ の固有値を $\lambda_1 \geq \lambda_2 \geq \ldots \geq \lambda_D$とし，これらに対応する単位固有ベクトルをそれぞれ $\mathbf{u}_1, \mathbf{u}_2,\ldots,\mathbf{u}_D$ とおくと，主成分分析で定まる線形変換を表す $H\times D$ 行列 $W_H$ は次のようになります．
$$
W_H = \begin{pmatrix}
\mathbf{u}_1^{\top} \\  \mathbf{u}_2^{\top} \\  \vdots \\ \mathbf{u}_H^{\top}
\end{pmatrix}
$$
このとき，変換後のデータ $\mathbf{y}_n = W_H\mathbf{x}_n$ の分散共分散行列がどうなるか考えてみましょう．

まず，$\mathbf{x}_n$の平均が $\mathbf{0}$ であることから，$\mathbf{y}_n$ の平均も $\mathbf{0}$ となります．式で書くと次の通りです．
$$
\frac{1}{N}\sum_{n=1}^{N}\mathbf{y}_n = \frac{1}{N}\sum_{n=1}^{N}W_H\mathbf{x}_{n} = W_H\left( \frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_n\right) = \mathbf{0}
$$
したがって，$\mathbf{y}_n$ の分散共分散行列を $V_y$ とおくと，$V_y = \frac{1}{N}\sum_{n=1}^N\mathbf{y}_n\mathbf{y}_n^{\top}$ となります．この式は，次のように変形できます．
$$
\begin{aligned}
V_y &= \frac{1}{N}\sum_{n=1}^N\left(W_H\mathbf{x}_n\right)\left(W_H\mathbf{x}_n\right)^{\top} \\
&= \frac{1}{N}\sum_{n=1}^N W_H\mathbf{x}_n\mathbf{x}_n^{\top}W_H^{\top}\\
&= W_H \left( \frac{1}{N}\sum_{n=1}^N \mathbf{x}_n\mathbf{x}_n^{\top} \right) W_H^{\top} \\
&= W_H V_x W_H^{\top}
\end{aligned}
$$

ここで，$W_H$ が $V_x$ の固有ベクトルをならべて作った行列だったことを思い出すと，この式は結局次のようになります（注）．
$$
V_y = \begin{pmatrix}
\lambda_1 & 0 & \cdots & 0\\
0 & \lambda_2 & \cdots & 0\\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & \lambda_H
\end{pmatrix}
$$

これより，変換後の特徴は次のような性質を持つことが分かります．
1. 変換後の特徴の各要素は互いに無相関である
1. $h$番目の特徴の分散は$V_x$ の $h$ 番目の固有値 $\lambda_h$ に等しい（$h = 1, 2, \ldots, H$）


※注: 少し過程を省略しましたが，行列の「固有値分解」と「対角化」の話で証明できます．


#### 数物情3次元データで確かめてみよう

いつもの「数学」「物理」「情報」の点数のデータで．

In [ ]:
# 数物情データを入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/PIP/mpi100-mac.csv
dfMPI = pd.read_csv('mpi100-mac.csv', index_col=0)
datMPI = dfMPI.to_numpy().astype(float)
#dfMPI

# 平均
Xm = np.mean(dfMPI.to_numpy(), axis=0)
print('平均:', Xm)

# 平均を引いて NumPy 配列にする
X = dfMPI.to_numpy() - Xm
N, D = X.shape
print('X.shape:', X.shape) # X は 100 x 3
print(X[:5, :]) # 最初の5人分を表示

次のセルを実行すると，平均を引いたデータの分散共分散行列を計算します．

In [ ]:
Vx = X.T @ X / N
Vx

上記の分散共分散行列の特徴を可視化するために，平均を引いたデータの「ペアプロット」を描いてみます．これは，データの特徴量の中から2つを選んでその散布図を描いたものです．対角には，その特徴量のヒストグラムが描かれています．


In [ ]:
dfX = pd.DataFrame(X, columns=['Math', 'Physics', 'Info'])
ax = seaborn.pairplot(dfX)
ax.set(xlim=(-40, 40))
ax.set(ylim=(-40, 40))
plt.show()

次に，主成分分析による変換をしたデータでも同じことをやってみましょう．

In [ ]:
# 分散共分散行列の固有値と固有ベクトルを求める（Xの特異値分解経由で）
_, sval, Vt = np.linalg.svd(X, full_matrices=False)
eval = sval**2/N
U = Vt
for d in range(D):
    print(f'{d+1}番目の固有値:{eval[d]:.2f}   固有ベクトル:', U[d, :])

# U を使って変換（次元は減らさずそのまま）
Y = X @ U.T
print(Y.shape)
print(Y[:5, :]) # 最初の5人分を表示

変換後のデータの分散共分散行列を求めてみると，次のようになります．対角要素の値は↑の固有値に等しくなっています．
一方，対角以外の要素は，数値計算の誤差があるためぴったり $0$ にはなっていませんが，絶対値の小さい値，$0$ に近いになっていることがわかるでしょう．

In [ ]:
Vy = Y.T @ Y / N
Vy

変換後のデータのペアプロットはこんなん．

In [ ]:
dfY = pd.DataFrame(Y, columns=['$y_1$', '$y_2$', '$y_3$'])
ax = seaborn.pairplot(dfY)
ax.set(xlim=(-40, 40))
ax.set(ylim=(-40, 40))
plt.show()

変換後のデータでは，特徴間の相関が $0$ となっていること，番号の小さい特徴量ほど分散が大きくなっている（ヒストグラムの広がり方に注目）ことが分かります．

---
### 再構成の誤差

#### 再構成したときの二乗誤差が最小となる線形変換になってるよ

主成分分析による次元削減のための変換
$\mathbf{y} = W_H\mathbf{x}$ では，$D$ 次元のベクトル $\mathbf{x}$ が $H$ 次元のベクトル $\mathbf{y}$ へ変換されます．
この $\mathbf{y}$ に左から行列 $W_H^{\top}$ をかけて
$$
\mathbf{z} = W_H^{\top}\mathbf{y} = W_H^{\top}W_H\mathbf{x}
$$
とすると，$\mathbf{z}$ は $D$ 次元ベクトルとなります．この操作のことを，次元削減したデータを元の次元数に戻すいう意味で，「再構成」と呼ぶことにします．

再構成には次のような性質があることが知られています：

変換行列 $W_H$ を用いた上記の再構成は，$H$次元へ次元削減して再構成するあらゆる線形変換の中で，学習データ $\mathbf{x}_n$ とその再構成 $\mathbf{z}_n$ との間の二乗誤差
$$
\sum_{n=1}^{N}\Vert\mathbf{z_n} - \mathbf{x}_{n}\Vert^2 
$$
が最小となるものの一つである．特に $H=D$ の場合は，再構成誤差は $0$ となる（完全に元に戻る）．

主成分分析では，元のデータの分散をなるべく保つように次元削減しており，かつ，小さい番号の小さい要素ほど分散が大きく，番号の大きい要素ほど分散が小さくなっています．したがって，番号の大きい方の要素を捨てて次元削減しても，元のデータの散らばりが損なわれず，「次元削減してから再構成したときの誤差が小さくなる」というわけです．

このような性質がありますので，主成分分析による次元削減の変換を行って得られる特徴量は，番号の小さいものほど重要な情報を保持していると考えることができます．

#### 数物情3次元データで実験してみよう

ふたたび「数学」「物理」「情報」の点数のデータで．



次のセルを実行すると，3次元の数物情データを `H` 次元へ変換してから再構成することができます．`H` の値を変えて，再構成した値や二乗誤差がどうなるか観察してみてね．

In [ ]:
#@title 数物情データを次元削減してから再構成
#@markdown `H` で次元削減後の次元数を指定
H = 1 #@param [1, 2, 3] {type: "raw"}

# 3次元からH次元へ
Y = X @ U[:H, :].T
# 再構成
Z = Y @ U[:H, :] 

# np.array を print する際の表示桁数を変更
precision = np.get_printoptions()['precision']
np.set_printoptions(precision=1)

# 最初の10人分を表示
print(f'元データ       {H}次元での再構成   二乗誤差')
for n in range(10):
    XX = X[n, :] + Xm
    ZZ = Z[n, :] + Xm
    sqe = np.sum((ZZ - XX)**2)
    print(f'{XX}  {ZZ}  {sqe:.2f}')

# np.array を print する際の表示桁数を元に戻しておく
np.set_printoptions(precision=precision)

---
### 主成分分析以外の次元削減手法



次元削減の手法には，主成分分析以外にも様々なものがあります．その一部をごく簡単に紹介しておきます．

<dl>
<dt>判別分析</dt> <dd> データが $K$ クラスに分類できると分かっているときに，その分類に最適な $K-1$ 次元特徴へと線形変換する手法．学習データにクラスラベルが必要なので，教師あり学習の手法といえる．</dd>
<dt>独立成分分析</dt> <dd> 変換後の特徴量が互いに独立となるように線形変換する手法．</dd>
</dl>

これら二つはいずれも線形変換による手法です．線形変換による手法は他にもたくさんあります．

一方，線形変換では実現できない次元削減のために，ニューラルネットワーク等を利用して非線形な次元削減を行う手法もいろいろあります．